# Determinando o Valor de um Carro Usando Dados Históricos
Rusty Bargain é um serviço de venda de carros usados que está desenvolvendo um aplicativo para atrair novos clientes. Nesse aplicativo, você pode descobrir rapidamente o valor de mercado do seu carro. Você tem acesso a dados históricos, especificações técnicas, versões de acabamento e preços. Precisamos construir o modelo para determinar o valor.

Rusty Bargain está interessado em:

- qualidade da predição
- velocidade da predição
- tempo necessário para o treinamento

# Sumário
1. [Iniciação](#1)
2. [Pré-processamento de Dados](#2)
3. [Treinamento de Modelos](#3)
  - [Linear Regression](#31)
  - [Decision Tree](#32)
  - [Random Forest](#33)
  - [CatBoost](#34)
  - [LightGBM](#35)
4. [Análise dos Modelos](#4)
5. [Conclusão](#5)

# Iniciação <a name='1'></a>

Importando as bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
import time
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
random_state=12345

In [2]:
try:
    data = pd.read_csv('autos.csv')
except:
    data = pd.read_csv('/datasets/autos.csv')

In [3]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Notamos algumas linhas com valores ausentes. Observemos a descrição das colunas numéricas:

In [4]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Observa-se que:
1. Há valores 0 em `Price`. Isso não faz sentido, portanto filtraremos.
2. Há valores de anos que de acordo com a história são impossíveis de estarem lá, já que o automóvel foi inventado em 1886 e estamos em 2023. A filtragem é necessária novamente.
3. `Power` 0. Isso não faz sentido em um carro. Novamente filtragem.
4. Os meses vão de 1 a 12, mas temos valores de 0 na coluna `RegistrationMonth`. Filtragem.
5. A coluna `NumberOfPictures` é completa de zeros. Descartaremos a coluna inteira, pois isso não é útil nesse contexto.

# Pré-processamento de Dados <a name='2'></a>

Verificaremos se há linhas duplicadas, caso haja alguma, iremos excluí-las.

In [5]:
data.duplicated().sum()

4

Processando os dados:

In [6]:
data = data[data['Price']!=0]
data = data[data['Power']!=0]
data = data[data['RegistrationMonth']!=0]
data = data[(1886 <= data['RegistrationYear']) & (data['RegistrationYear'] <= 2021)]
data = data.drop('NumberOfPictures', axis=1)
data['Repaired'].fillna(value='unknown', inplace=True)
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253811 entries, 2 to 354368
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        253811 non-null  object
 1   Price              253811 non-null  int64 
 2   VehicleType        253811 non-null  object
 3   RegistrationYear   253811 non-null  int64 
 4   Gearbox            253811 non-null  object
 5   Power              253811 non-null  int64 
 6   Model              253811 non-null  object
 7   Kilometer          253811 non-null  int64 
 8   RegistrationMonth  253811 non-null  int64 
 9   FuelType           253811 non-null  object
 10  Brand              253811 non-null  object
 11  Repaired           253811 non-null  object
 12  DateCreated        253811 non-null  object
 13  PostalCode         253811 non-null  int64 
 14  LastSeen           253811 non-null  object
dtypes: int64(6), object(9)
memory usage: 31.0+ MB


Processamento finalizado. Agora prepararemos os conjuntos de dados. Treinaremos os modelos Linear Regression (modelo fraco), Decision Tree Regressor, Random Forest Regressor, CatBoost Regressor e LightGBM Regressor. Os três primeiros modelos exigirão que codifiquemos as características categóricas antes do treinamento. Os 2 últimos não precisam de codificação prévia, pois possuem codificadores integrados, portanto faremos 2 cópias dos dados limpos: 1 que não está codificada (que será usada com CatBoost e LightGBM), e outra que será codificada para os outros modelos. Primeiro, os dados que permanecerão não codificados:

In [7]:
data_gbm = data.copy()
data_gbm.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1, inplace=True) #colunas com formato data-hora
data_gbm.reset_index(drop=True, inplace=True)

f_gbm = data_gbm.drop('Price', axis=1)
t_gbm = data_gbm['Price']

f_train_gbm, f_test_gbm, t_train_gbm, t_test_gbm=train_test_split(f_gbm, t_gbm, test_size=0.2,
                                                                  random_state=random_state)

print(f_train_gbm.shape)
print(t_train_gbm.shape)
print(f_test_gbm.shape)
print(t_test_gbm.shape)

(203048, 11)
(203048,)
(50763, 11)
(50763,)


Criaremos uma lista com as colunas que queremos codificar:

In [8]:
cat_feat = ['VehicleType', 'RegistrationYear', 'Gearbox', 'Model', 'RegistrationMonth',
           'FuelType', 'Brand', 'Repaired', 'PostalCode']

In [9]:
data_enc = data.copy()
encoder = OrdinalEncoder()
data_enc[cat_feat] = encoder.fit_transform(data_enc[cat_feat])

Preparando e dividindo os dados codificados

In [10]:
data_enc.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1, inplace=True)
data_enc.reset_index(drop=True, inplace=True)
f_enc = data_enc.drop('Price', axis=1)
t_enc = data_enc['Price']
f_train_enc, f_test_enc, t_train_enc, t_test_enc = train_test_split(f_enc, t_enc, test_size=0.2,
                                                                    random_state=random_state)
print(f_train_enc.shape)
print(t_train_enc.shape)
print(f_test_enc.shape)
print(t_test_enc.shape)

(203048, 11)
(203048,)
(50763, 11)
(50763,)


Pré-processamento terminado. Preparamos, portanto, os dados codificados e não codificados para o treinamento do modelo

# Treinamento dos Modelos <a name='3'></a>

Primeiro criaremos uma função que calcule o RMSE para torná-la nossa métrica de avaliação para os modelos.

In [11]:
def rmse(pred, target):
    pred = np.array(pred)
    target = np.array(target)
    error = pred - target
    sq_error = error ** 2
    msq_error = sq_error.mean()
    score = msq_error ** 0.5
    return score
scorer = make_scorer(rmse, greater_is_better=False)

## Linear Regression<a name='31'></a>

Treinaremos um modelo de Regressão Linear como o modelo base, ou seja, o valor da métrica de avaliação (RMSE) que obteremos dele será aquele que os outros modelos devem se esforçar para superar. Usaremos os dados codificados. Obtendo uma pontuação de validação cruzada:

In [12]:
LR = LinearRegression()
LR_score = cross_val_score(LR, f_train_enc, t_train_enc, scoring=scorer, cv=5)
print(LR_score.mean())

-3336.4188026671195


Treinando o modelo e temporizando o tempo para o mesmo.

In [13]:
LR = LinearRegression()
%time LR.fit(f_train_enc, t_train_enc)

Wall time: 67.8 ms


LinearRegression()

Predições:

In [14]:
%time lr_pred = LR.predict(f_test_enc)

Wall time: 4.99 ms


RMSE:

In [15]:
lr_rmse = rmse(t_test_enc, lr_pred)
lr_rmse

3340.919561096338

O valor a ser superado será o RMSE de `3340.91`

## Decision Tree Regressor<a name='32'></a>

Em seguida, realizaremos o ajuste de hiperparâmetro (com o hiperparâmetro max_depth) para encontrar o melhor valor para defini-lo no momento de treinar. Escolheremos o hiperparâmetro que obtém a melhor pontuação de validação cruzada

In [16]:
for depth in range(1, 16):
    DTR = DecisionTreeRegressor(max_depth=depth, random_state=random_state)
    DTR_score = cross_val_score(DTR, f_train_enc, t_train_enc, scoring=scorer, cv=5)
    print('Max_depth', depth, 'score:', DTR_score.mean())

Max_depth 1 score: -3591.779208179859
Max_depth 2 score: -3144.1767270742685
Max_depth 3 score: -2787.5732866460216
Max_depth 4 score: -2537.915451395532
Max_depth 5 score: -2367.5365634955133
Max_depth 6 score: -2257.4949498165524
Max_depth 7 score: -2159.6167183099237
Max_depth 8 score: -2078.937395240203
Max_depth 9 score: -2010.3866104451256
Max_depth 10 score: -1950.8771669216178
Max_depth 11 score: -1910.414754693075
Max_depth 12 score: -1883.759903175952
Max_depth 13 score: -1879.836681144283
Max_depth 14 score: -1891.4998381214039
Max_depth 15 score: -1906.1482830792634


A melhor pontuação foi com `max_depth=13`. Treinaremos o modelo e obteremos previsões com o mesmo juntamente com os processos cronometrados

In [17]:
DTR = DecisionTreeRegressor(max_depth=13, random_state=random_state)
%time DTR.fit(f_train_enc, t_train_enc)

Wall time: 1.01 s


DecisionTreeRegressor(max_depth=13, random_state=12345)

In [18]:
%time DTR_pred = DTR.predict(f_test_enc)

Wall time: 17.9 ms


RMSE:

In [19]:
DTR_rmse = rmse(t_test_enc, DTR_pred)
DTR_rmse

1861.0578704978764

O RMSE é `1861.05`, muito melhor que o anterior.

## Random Forest Regressor<a name='33'></a>

Realizaremos o ajuste de hiperparâmetros com 2 deles: `max_depth` e `n_estimators`. Depois de realizá-lo, o melhor modelo obteu `n_estimators=70` e `max_depth=24`

In [20]:
for depth in range(20, 26):
    RFR = RandomForestRegressor(n_estimators=70, max_depth=depth, random_state=random_state)
    RFR_score = cross_val_score(RFR, f_train_enc, t_train_enc, scoring=scorer, cv=5)
    print('Max_depth', depth, 'score:', RFR_score.mean())

Max_depth 20 score: -1551.6923307505144
Max_depth 21 score: -1549.4629438418308
Max_depth 22 score: -1548.566238415244
Max_depth 23 score: -1548.8232450598139
Max_depth 24 score: -1547.8265334168775
Max_depth 25 score: -1547.9100710431164


Treinando e testando o modelo com o tempo.

In [21]:
RFR = RandomForestRegressor(n_estimators=70, max_depth=24, random_state=random_state)
%time RFR.fit(f_train_enc, t_train_enc)

Wall time: 1min 4s


RandomForestRegressor(max_depth=24, n_estimators=70, random_state=12345)

In [22]:
%time RFR_pred = RFR.predict(f_test_enc)

Wall time: 1.69 s


RMSE:

In [23]:
RFR_rmse = rmse(t_test_enc, RFR_pred)
RFR_rmse

1523.7475976086446

RMSE de `1523.74`, uma melhora significativa.

## CatBoost Regressor<a name='34'></a>

Para o CatBoost Regressor, não é necessário uma codificação prévia. Agora, ajustaremos usando diferentes hiperparâmetros e encontraremos os melhores usando GridSearchCV.

In [24]:
CBR = CatBoostRegressor()
parameters = {'depth': [6, 8, 10],
             'learning_rate': [0.5, 0.1],
             'l2_leaf_reg': [2, 4],
             'iterations': [10, 50],
             'loss_function': ['RMSE'],
             'random_seed': [random_state]}
#o dicionário de hiperparâmetros que será percorrido no GridSearch
grid = GridSearchCV(estimator=CBR, param_grid=parameters, scoring=scorer, cv=3, n_jobs=-1, verbose=0)
#percorre os parâmetros para obter os melhores hiperparâmetros para o treinamento do modelo
grid.fit(f_train_gbm, t_train_gbm, cat_features=cat_feat)
best_param = grid.best_params_

0:	learn: 3045.4364562	total: 304ms	remaining: 14.9s
1:	learn: 2317.9833730	total: 421ms	remaining: 10.1s
2:	learn: 2018.5864758	total: 567ms	remaining: 8.88s
3:	learn: 1881.5349365	total: 709ms	remaining: 8.15s
4:	learn: 1825.1594403	total: 847ms	remaining: 7.62s
5:	learn: 1778.3627126	total: 967ms	remaining: 7.09s
6:	learn: 1749.1081322	total: 1.09s	remaining: 6.68s
7:	learn: 1725.8749630	total: 1.2s	remaining: 6.28s
8:	learn: 1698.6889417	total: 1.3s	remaining: 5.93s
9:	learn: 1681.6086935	total: 1.41s	remaining: 5.66s
10:	learn: 1664.9021222	total: 1.52s	remaining: 5.39s
11:	learn: 1655.8441379	total: 1.63s	remaining: 5.17s
12:	learn: 1642.8163726	total: 1.74s	remaining: 4.96s
13:	learn: 1628.3678863	total: 1.85s	remaining: 4.76s
14:	learn: 1621.4065612	total: 1.96s	remaining: 4.57s
15:	learn: 1614.0514458	total: 2.07s	remaining: 4.4s
16:	learn: 1609.8892278	total: 2.17s	remaining: 4.22s
17:	learn: 1604.4348659	total: 2.28s	remaining: 4.06s
18:	learn: 1599.0858178	total: 2.38s	rema

Melhor conjunto de hiperparâmetros para o modelo:

In [25]:
print('Melhor pontuação de todos os parâmetros pesquisados', grid.best_score_)
print('Melhores parâmetros:', best_param)

Melhor pontuação de todos os parâmetros pesquisados -1602.8797091324839
Melhores parâmetros: {'depth': 10, 'iterations': 50, 'l2_leaf_reg': 2, 'learning_rate': 0.5, 'loss_function': 'RMSE', 'random_seed': 12345}


Treinando CatBoost Regressor usando as melhores configurações de hiperparâmetros obtidas cronometrando o processo:

In [26]:
CBR = CatBoostRegressor(depth=best_param['depth'],
                       iterations=best_param['iterations'],
                       l2_leaf_reg=best_param['l2_leaf_reg'],
                       learning_rate=best_param['learning_rate'],
                       loss_function='RMSE', random_seed=random_state)
%time CBR.fit(f_train_gbm, t_train_gbm, cat_features=cat_feat, verbose=False, plot=False)

Wall time: 7 s


Predições:

In [27]:
%time CBR_pred = CBR.predict(f_test_gbm)

Wall time: 204 ms


RMSE:

In [28]:
CBR_rmse = rmse(t_test_gbm, CBR_pred)
CBR_rmse

1608.753392842063

RMSE de `1608.75`. Uma leve decaída em relação ao precedente.

## LightGBM Regressor<a name='35'></a>

Este também não requer codificação prévia, logo, estaremos realizando o ajuste de hiperparâmetros semelhante ao feito para o CatBoost, exceto pelo fato que estaremos lidando com diferentes hiperparâmetros. No LightGBM as características categóricas devem ser do tipo `category` antes de colocá-las no LightGBM. Dessarte, precisaremos fazer isso tanto para o conjunto de treinamento quanto para o conjunto de teste

In [29]:
for c in cat_feat:
    f_train_gbm[c] = f_train_gbm[c].astype('category')

model = LGBMRegressor()
parameters = {'num_leaves': [10, 20, 30],
             'learning_rate': [0.5, 0.1],
             'n_estimators': [10, 20],
             'random_state': [random_state],
            'objective': ['rmse']}
grid = GridSearchCV(estimator=model, param_grid=parameters, scoring=scorer, cv=3, n_jobs=-1)
grid.fit(f_train_gbm, t_train_gbm)
best_param = grid.best_params_

Buscando a melhor configuração de hiperparâmetros:

In [30]:
print('Melhor pontuação de todos os parâmetros pesquisados', grid.best_score_)
print('Melhores parâmetros:', best_param)

Melhor pontuação de todos os parâmetros pesquisados -1678.5078420435445
Melhores parâmetros: {'learning_rate': 0.5, 'n_estimators': 20, 'num_leaves': 30, 'objective': 'rmse', 'random_state': 12345}


Treinando e testando o modelo LightGBM usando as melhores configurações, enquanto o processo é cronometrado

In [31]:
lgbm = LGBMRegressor(learning_rate=0.5,
                    n_estimators=20,
                    num_leaves=30,
                    objective='rmse', random_state=random_state)
%time lgbm.fit(f_train_gbm, t_train_gbm)

Wall time: 621 ms


LGBMRegressor(learning_rate=0.5, n_estimators=20, num_leaves=30,
              objective='rmse', random_state=12345)

In [32]:
for c in cat_feat:
    f_test_gbm[c] = f_test_gbm[c].astype('category')

%time lgbm_pred = lgbm.predict(f_test_gbm)

Wall time: 69.8 ms


RMSE:

In [33]:
lgbm_rmse = rmse(t_test_gbm, lgbm_pred)
lgbm_rmse

1674.8093282242774

RMSE de `1674.80`

# Análise de Modelos <a name='4'></a>

Preparação de uma tabela mostrando os diferentes modelos e seus tempos de treinamento e previsão (em milissegundos) e RMSEs:

In [35]:
index = ['LR', 'DTR', 'RFR', 'CB', 'LGBM']
summary = pd.DataFrame(data={'training_time(ms)': [68, 1010, 64000, 7000, 621],
                             'prediction_time(ms)': [5, 18, 1690, 204, 70],
                             'RMSE': [3340, 1861, 1523, 1608, 1674]},
                      index=index)
summary

,training_time(ms),prediction_time(ms),RMSE
LR,68,5,3340
DTR,1010,18,1861
RFR,64000,1690,1523
CB,7000,204,1608
LGBM,621,70,1674


Observações:
1. Linear Regression teve o melhor tempo de treinamento (68 ms), enquanto o pior vai para Random Forest (64000 ms)
2. Linear Regression teve o melhor tempo de previsão (5 ms), enquanto o pior vai para a Random Forest (1690 ms)
3. Random Forest teve o melhor RMSE (1523), enquanto o pior vai para a Regressão Linear (3340)

# Conclusão <a name='5'></a>

Processamos e preparamos os dados com sucesso e os usamos para treinar modelos. Embora o modelo Random Forest tenha o maior RMSE (1523), o custo em termos de treinamento e tempo de previsão é muito grande. O CatBoost Regressor demora menos da metade desse tempo e obtém um RMSE de 1608, uma diferença de apenas 85. Porém, considerando o tempo de treinamento, a melhor escolha é o LightGBM dado que possui um ótimo balanço entre ambos, perdendo apenas 66 pontos no RMSE (1674). Portanto, a recomendação final é o modelo `LightGBM`.